In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lastly/collect_preprocessed_dataset - collect_preprocessed_dataset.csv


In [4]:
!pip install tensorflow scikit-learn pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 8.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


2025-12-13 07:10:51.735464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765609851.939935      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765609852.008242      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
df = pd.read_csv("/kaggle/input/lastly/collect_preprocessed_dataset - collect_preprocessed_dataset.csv")

In [7]:
df.columns

Index(['Data', 'Love', 'Joy', 'Anger', 'Surprise', 'Sadness', 'Fear', 'Hate',
       'topic', 'Domain'],
      dtype='object')

In [8]:
emotion_cols = ['Love', 'Joy', 'Anger', 'Surprise', 'Sadness', 'Fear', 'Hate']

texts = df['Data'].astype(str).values
labels = df[emotion_cols].values

In [9]:
MAX_WORDS = 30000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=MAX_LEN, padding='post')

y = labels


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [11]:
model = Sequential([
    # 1️⃣ Embedding layer
    Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN),

    # 2️⃣ BiLSTM layer
    Bidirectional(LSTM(64, return_sequences=False)),

    # 3️⃣ Dense hidden layer
    Dense(64, activation='relu'),
    Dropout(0.5),

    # 4️⃣ Output layer (Multi-label)
    Dense(len(emotion_cols), activation='sigmoid')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1765609867.864474      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1765609867.865152      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [12]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=32
)


Epoch 1/5


I0000 00:00:1765609872.608773     120 cuda_dnn.cc:529] Loaded cuDNN version 90300


626/626 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.2456 - loss: 0.4325 - val_accuracy: 0.5119 - val_loss: 0.2882
Epoch 2/5
626/626 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.5998 - loss: 0.2520 - val_accuracy: 0.5712 - val_loss: 0.2643
Epoch 3/5
626/626 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.7606 - loss: 0.1703 - val_accuracy: 0.5924 - val_loss: 0.2652
Epoch 4/5
626/626 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.8358 - loss: 0.1186 - val_accuracy: 0.5919 - val_loss: 0.3000
Epoch 5/5
626/626 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.8813 - loss: 0.0851 - val_accuracy: 0.5978 - val_loss: 0.3581


In [14]:
y_pred = model.predict(X_test)
y_pred_bin = (y_pred > 0.5).astype(int)

print(classification_report(y_test, y_pred_bin, target_names=emotion_cols))


174/174 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
              precision    recall  f1-score   support

        Love       0.72      0.60      0.66       877
         Joy       0.54      0.54      0.54       873
       Anger       0.55      0.50      0.52       856
    Surprise       0.80      0.74      0.77       827
     Sadness       0.51      0.53      0.52       809
        Fear       0.83      0.73      0.78       921
        Hate       0.68      0.64      0.66       867

   micro avg       0.66      0.61      0.63      6030
   macro avg       0.66      0.61      0.63      6030
weighted avg       0.66      0.61      0.64      6030
 samples avg       0.61      0.61      0.61      6030



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
def predict_emotions(text):
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=MAX_LEN, padding='post')
    probs = model.predict(pad)[0]

    return {
        emotion_cols[i]: float(probs[i])
        for i in range(len(emotion_cols))
        if probs[i] > 0.5
    }

print(predict_emotions("ami onek khushi ar valobashi"))
print(predict_emotions("amar mon khub kharap lagche aj"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
{'Love': 0.9333289861679077}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
{'Sadness': 0.7664315700531006}


In [17]:
def predict_emotions_topk(text, k=2):
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=MAX_LEN, padding='post')

    probs = model.predict(pad)[0]

    emotion_scores = list(zip(emotion_cols, probs))
    emotion_scores.sort(key=lambda x: x[1], reverse=True)

    return emotion_scores[:k]


In [18]:
print(predict_emotions_topk("ami onek khushi ar valobashi", k=3))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[('Love', 0.933329), ('Joy', 0.037743486), ('Sadness', 0.029820226)]
